# 🚀 ConnectIT Cloud Node (Google Colab)

Run a ConnectIT P2P node directly from Google Colab.

### 📝 Instructions
1. **Install Dependencies**: Run Cell 1.
2. **Choose Tunnel**: Run Cell 2 for Bore Tunnel (No Account, High Reliability).
3. **Run Node**: Run the final cell to start the node.

In [4]:
# @title 1. Install Dependencies & Setup
# @markdown This cell installs ConnectIT, dependencies, and setup tools.

!git clone https://github.com/Chatit-cloud/BEE2BEE.git
#!rm -r BEE2BEE 
%cd BEE2BEE
!pip install -e .
#!pip install riche

Cloning into 'BEE2BEE'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 258 (delta 109), reused 231 (delta 82), pack-reused 0 (from 0)
Receiving objects: 100% (258/258), 237.80 KiB | 3.01 MiB/s, done.
Resolving deltas: 100% (109/109), done.
/content/BEE2BEE
Obtaining file:///content/BEE2BEE
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for connectit (pyproject.toml) ... done
  Created wheel for connectit: filename=connectit-0.1.0-0.editable-py3-none-any.whl size=7139 sha256=9b76fff00571e96e1a186ba2547f9e47bf03cbd01f19621dbfb581a7e3a09c02
  Stored in directory: /tmp/pip-ephem-wheel-cache-gp244dcz/wheels/8f/eb/64/dfd48ea2148c2ea6e33b8ab399a5d9ab9aaa3f6ca702fd52ca
Successfully built connectit
  Attempting

In [5]:
# @title 3. Configure Hybrid Tunnels (Cloudflare API + Bore P2P)
# @markdown **Why Hybrid?**
# @markdown - **API (HTTP)**: Uses **Cloudflare Tunnel** for maximum stability and speed.
# @markdown - **P2P (WebSocket)**: Uses **Bore Tunnel** for raw TCP/WS support without landing pages.

import os
import time
import subprocess
import threading
import re

API_PORT = 4002
P2P_PORT = 4003

# 1. Install Cloudflared
if not os.path.exists("cloudflared"):
    print("📥 Downloading Cloudflared...")
    !wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared

# 2. Install Bore
if not os.path.exists("bore"):
    print("📥 Downloading Bore...")
    !wget -q https://github.com/ekzhang/bore/releases/download/v0.5.0/bore-v0.5.0-x86_64-unknown-linux-musl.tar.gz
    !tar -xzf bore-v0.5.0-x86_64-unknown-linux-musl.tar.gz
    !chmod +x bore
    !rm bore-v0.5.0-x86_64-unknown-linux-musl.tar.gz

# Clear logs
!rm -f tunnel_*.log

def start_cloudflare_tunnel(port, log_file):
    print(f"🔄 Starting Cloudflare Tunnel for API (Port {port})...")
    cmd = f"./cloudflared tunnel --url http://localhost:{port} > {log_file} 2>&1"
    subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def start_bore_tunnel(port, log_file):
    print(f"🔄 Starting Bore Tunnel for P2P (Port {port})...")
    cmd = f"./bore local {port} --to bore.pub > {log_file} 2>&1"
    subprocess.Popen(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Kill old processes
!pkill cloudflared
!pkill bore

# Start Tunnels in Background
threading.Thread(target=start_cloudflare_tunnel, args=(API_PORT, "tunnel_api.log"), daemon=True).start()
threading.Thread(target=start_bore_tunnel, args=(P2P_PORT, "tunnel_p2p.log"), daemon=True).start()

print("⏳ Waiting for tunnels to initialize...")
time.sleep(8) 

# Parse URLs
api_url = None
p2p_url = None

# Find Cloudflare URL
if os.path.exists("tunnel_api.log"):
    with open("tunnel_api.log", "r") as f:
        content = f.read()
        # Regex for trycloudflare.com
        match = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", content)
        if match:
            api_url = match.group(0)

# Find Bore URL
if os.path.exists("tunnel_p2p.log"):
    with open("tunnel_p2p.log", "r") as f:
        content = f.read()
        # listening at bore.pub:xxxxx
        for line in content.splitlines():
             if "listening at bore.pub" in line:
                 parts = line.split("bore.pub:")
                 if len(parts) > 1:
                     port_num = parts[1].strip()
                     p2p_url = f"ws://bore.pub:{port_num}"

print("\n" + "="*60)
print("🔗  Active Hybrid Tunnel Configuration")
print("="*60)

if api_url:
    print(f"✅ API Tunnel (Cloudflare): {api_url}")
    print(f"   (Use this URL in your Browser or Desktop App)")
else:
    print("❌ API Tunnel: Not found yet. Check tunnel_api.log")

if p2p_url:
    print(f"✅ P2P Tunnel (Bore):       {p2p_url}")
    print(f"   (Use this URL for 'bootstrap_url')")
else:
    print("❌ P2P Tunnel: Not found yet. Check tunnel_p2p.log")

print("="*60 + "\n")

# Save config for next cell
with open("tunnel_config.txt", "w") as f:
    if api_url: f.write(f"API Tunnel: {api_url}\n")
    if p2p_url: f.write(f"P2P (WS) Tunnel: {p2p_url}\n")


📥 Downloading Bore...
🔄 Starting Tunnel for API (HTTP) (Port 4002)...
🔄 Starting Tunnel for P2P (WS) (Port 4003)...

✅ API (HTTP) Tunnel: http://bore.pub:2319

✅ P2P (WS) Tunnel: ws://bore.pub:21253


In [ ]:
# 4. Run ConnectIT Node
import os
import re
import time
from urllib.parse import urlparse

print("🔍 Analyzing Tunnel Configuration...")

p2p_url = None

# Try to read from config file first
if os.path.exists("tunnel_config.txt"):
    with open("tunnel_config.txt", "r") as f:
        lines = f.readlines()
        for line in reversed(lines):
            if "P2P (WS) Tunnel:" in line:
                # format: ✅ P2P (WS) Tunnel: ws://bore.pub:xxxxx
                parts = line.split("Tunnel: ")
                if len(parts) > 1:
                    p2p_url = parts[1].strip()
                    break

if p2p_url:
    print(f"✅ Found P2P Tunnel: {p2p_url}")
    try:
        parsed = urlparse(p2p_url)
        os.environ["CONNECTIT_ANNOUNCE_HOST"] = parsed.hostname
        os.environ["CONNECTIT_ANNOUNCE_PORT"] = str(parsed.port)
        # Legacy fallback
        os.environ["CONNECTIT_PUBLIC_ADDR"] = f"{parsed.hostname}:{parsed.port}"
        print(f"   → Announce Address set to: {parsed.hostname}:{parsed.port}")
    except Exception as e:
        print(f"⚠️ Failed to parse URL: {e}")
else:
    print("⚠️ Could not find P2P Tunnel URL. Node may announce local IP (unreachable).")

print("\n🚀 Starting Node...")
!python -m connectit api

🔍 Analyzing Tunnel Configuration...
✅ Found P2P Tunnel: ws://bore.pub:21253
   → Announce Address set to: bore.pub:21253

🚀 Starting Node...
🚀 Starting Main Point API on http://127.0.0.1:4002
INFO:     Started server process [16318]
INFO:     Waiting for application startup.
[16:11:29] Attempting UPnP Port Mapping for port 4003...      ]8;id=131510;file:///content/BEE2BEE/connectit/p2p_runtime.py\p2p_runtime.py]8;;\:]8;id=332982;file:///content/BEE2BEE/connectit/p2p_runtime.py#125\125]8;;\
           UPnP Mapping Failed (or not supported by router).  ]8;id=388930;file:///content/BEE2BEE/connectit/p2p_runtime.py\p2p_runtime.py]8;;\:]8;id=575497;file:///content/BEE2BEE/connectit/p2p_runtime.py#133\133]8;;\
           Manual forwarding needed.                                            
           P2P Node Started at ws://172.28.0.12:4003          ]8;id=151346;file:///content/BEE2BEE/connectit/p2p_runtime.py\p2p_runtime.py]8;;\:]8;id=965088;file:///content/BEE2BEE/con

INFO:     Shutting down
INFO:     Finished server process [16318]
^C
